# Techlabs: Matchmaking Process

Criteria for Match-making:
- Age: people within a certain age range (categories:below 40; 40-50 or 50-65, above 65 - weight 1 point
- Location: live in the same area in Copenhagen (Vesterbro, Valbey, Norrebro, Osterbro, Amager) - weight 0.5 points
- Interests and Hobbies: coffee / tea, walks, cooking, reading, concerts, sport, theater - 1 point for each common interest
- Number of matches: how many matches do they want to get (1, 2, 3)

The code you provided defines two classes, User and MatchMaker.

The User class has a constructor that takes in the following parameters:
- name: a string representing the name of the user
- age: an integer representing the age of the user
- interests: a list of strings representing the interests of the user
- location: a string representing the location of the user
- num_matches: an integer representing the number of matches the user wants to receive

The MatchMaker class has a constructor that takes in a list of User objects and creates matches between them based on their interests, location, and age. It also has several functions:
- match_making_names(user): takes in a User object and returns a list of User objects that are a match for that user
- match_making_scores(user): takes in a User object and returns a list of scores for the matches for that user
- calculate_matches(): calculates the matches for all users in the users list
- remove_match(user, other_user): takes in two User objects and removes the match between them
- delete_account(user): takes in a User object and removes it from the users list and also removes any matches with that user.
- add_account(user): takes in a User object and adds it to the users list and also recalculates all the matches.
- all_matches() : Returns all the matches calculated by the match_maker.The MatchMaker class has a constructor that takes in a list of User objects and creates matches between them based on their interests, location, and age. It also has several functions:

In [176]:
class User:
    # Class to create a new user
    def __init__(self, name, age, interests, location, num_matches):
        self.name = name
        self.age = age
        self.interests = interests
        self.location = location
        self.num_matches = num_matches
    
    def __repr__(self):
        return f"{self.name}"
        

class MatchMaker:
    # Class that creates matches
    def __init__(self, users):
        self.users = users
        self.matches = {}  # Initialize empty dictionary to store matches
        self.score = {} # Initialize empty dictionary to store scores of matches
        self.calculate_matches()  # Calculate matches for all users
        

    def match_making_names(self, user):
        if user in self.matches:
            return self.matches[user] # Return the stored matches for the user
        else:
            return []
        
    def match_making_scores(self,user):
        if user in self.matches:
            return self.score[user] # Return the stored scores for the user
        else:
            return []
    
    def calculate_matches(self):

        # Loop through all users in the database
        for user in self.users:
            # Initialize empty list to store matches for the current user
            user_matches = []
            for other_user in self.users:
                # Skip the current user
                if other_user == user:
                    continue

                # Calculate match score:
                # 1. Age
                age_diff = abs(user.age - other_user.age)
                if age_diff <= 5:
                    age_score = 1
                elif age_diff <= 10:
                    age_score = 0.5
                else:
                    age_score = 0
        
                # 2. Common Interest 
                common_interests = len(set(user.interests) & set(other_user.interests))

                # 3. Same Location
                if user.location == other_user.location:
                    location_score = 0.5
                else:
                    location_score = 0

                # Calculate score by addition
                match_score = age_score + common_interests + location_score

                # If match score is high enough, add other_user to the matches list
                if match_score > 0.5:
                    user_matches.append((other_user, match_score))

            # Sort the matches list by score in descending order
            user_matches.sort(key=lambda x: x[1], reverse=True)
            self.matches[user] = [match[0] for match in user_matches[:user.num_matches]]
            self.score[user] = [match[1] for match in user_matches[:user.num_matches]]
        
    def remove_match(self, user, other_user):
            # Find the current matches for user
            if user in self.matches:
                matches = self.matches[user]
                # If other_user is in the matches list, remove it
                if other_user in matches:
                    index = matches.index(other_user)
                    matches.remove(other_user)
                    self.matches[user] = matches

                    score = self.score[user]
                    score.pop(index)
                    self.score[user] = score
            # remove the match from the other_user
            if other_user in self.matches:
                other_matches = self.matches[other_user]
                if user in other_matches:
                    index = other_matches.index(user)
                    other_matches.remove(user)
                    self.matches[other_user] = other_matches

                    other_score = self.score[other_user]
                    other_score.pop(index)
                    self.score[other_user] = other_score

    def delete_account(self, user):
        # Remove user from the users list
        self.users.remove(user)
        
        # Remove user from the matches dictionary
        if user in self.matches:
            del self.matches[user]
            del self.score[user]
    
        # Loop through all users in the users list
        for other_user in self.users:
            # Remove user from other_user's matches list
            self.remove_match(other_user, user)
            
        # Recalculate matches for all users after removing the deleted user
        self.calculate_matches()
            
    def add_user(self, new_user):
        self.users.append(new_user)
        self.calculate_matches()
        
    def all_matches(self):
        return self.matches, self.score

In [177]:
# Create 5 users
user_Alice = User("Alice", 45, ["coffee", "walks", "reading", "theater"], "Vesterbro", 2)
user_Bob = User("Bob", 55, ["tea", "cooking", "sport", "theater"], "Valby", 3)
user_Charlie = User("Charlie", 60, ["coffee", "walks", "reading", "concerts"], "Norrebro", 1)
user_David = User("David", 75, ["tea", "cooking", "sport", "theater"], "Osterbro", 3)
user_Emily = User("Emily", 40, ["coffee", "walks", "reading", "concerts"], "Amager", 2)

# Create a list of users
users_list = [user_Alice, user_Bob, user_Charlie, user_David, user_Emily]

# Create a MatchMaker object
match_maker = MatchMaker(users_list)
print(match_maker.all_matches())

({Alice: [Emily, Charlie], Bob: [David, Alice, Charlie], Charlie: [Emily], David: [Bob, Alice], Emily: [Alice, Charlie]}, {Alice: [4, 3], Bob: [4, 1.5, 1], Charlie: [4], David: [4, 1], Emily: [4, 4]})


In [178]:
# Print matches for user1
print("Matches for Alice: ", match_maker.match_making_names(user_Alice))
print("Scores for Alice: ", match_maker.match_making_scores(user_Alice))

# Print matches for user2
print("Matches for Bob: ", match_maker.match_making_names(user_Bob))
print("Match scores for Bob: ", match_maker.match_making_scores(user_Bob))

# Print matches for user3
print("Matches for Charlie: ", match_maker.match_making_names(user_Charlie))
print("Scores for Charlie: ", match_maker.match_making_scores(user_Charlie))

# Print matches for user4
print("Matches for David: ", match_maker.match_making_names(user_David))
print("Match scores for David: ", match_maker.match_making_scores(user_David))

# Print matches for user5
print("Matches for Emily: ", match_maker.match_making_names(user_Emily))
print("Match scores for Emily: ", match_maker.match_making_scores(user_Emily))

Matches for Alice:  [Emily, Charlie]
Scores for Alice:  [4, 3]
Matches for Bob:  [David, Alice, Charlie]
Match scores for Bob:  [4, 1.5, 1]
Matches for Charlie:  [Emily]
Scores for Charlie:  [4]
Matches for David:  [Bob, Alice]
Match scores for David:  [4, 1]
Matches for Emily:  [Alice, Charlie]
Match scores for Emily:  [4, 4]


In [179]:
# Add new user to the system
print("Start: ", match_maker.all_matches())
user_Fiona = User("Fiona", 55, ["coffee", "cooking", "reading"], "Amager", 2)
match_maker.add_user(user_Fiona)
print("After: ", match_maker.all_matches())

Start:  ({Alice: [Emily, Charlie], Bob: [David, Alice, Charlie], Charlie: [Emily], David: [Bob, Alice], Emily: [Alice, Charlie]}, {Alice: [4, 3], Bob: [4, 1.5, 1], Charlie: [4], David: [4, 1], Emily: [4, 4]})
After:  ({Alice: [Emily, Charlie], Bob: [David, Fiona, Alice], Charlie: [Emily], David: [Bob, Alice, Fiona], Emily: [Alice, Charlie], Fiona: [Charlie, Alice]}, {Alice: [4, 3], Bob: [4, 2, 1.5], Charlie: [4], David: [4, 1, 1], Emily: [4, 4], Fiona: [3, 2.5]})


In [180]:
match_maker.delete_account(user_Fiona)
print(match_maker.all_matches())

({Alice: [Emily, Charlie], Bob: [David, Alice, Charlie], Charlie: [Emily], David: [Bob, Alice], Emily: [Alice, Charlie]}, {Alice: [4, 3], Bob: [4, 1.5, 1], Charlie: [4], David: [4, 1], Emily: [4, 4]})


In [181]:
# Remove match between user2 and user3
match_maker.remove_match(user_David, user_Bob)

# Print matches for user4
print("Matches for David: ", match_maker.match_making_names(user_David))
print("Match scores for David: ", match_maker.match_making_scores(user_David))

# Print matches for user2
print("Matches for Bob: ", match_maker.match_making_names(user_Bob))
print("Match scores for Bob: ", match_maker.match_making_scores(user_Bob))

Matches for David:  [Alice]
Match scores for David:  [1]
Matches for Bob:  [Alice, Charlie]
Match scores for Bob:  [1.5, 1]
